In [ ]:
from google.colab import drive
drive.mount('/content/drive')

The Code begins with importing of all the necessary library file.


In [ ]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
import datetime, os
import math
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

#library required for image preprocessing
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from  keras.applications.xception import preprocess_input

#library required for finding mean absolute error.
from keras.metrics import mean_absolute_error

#Some Important machine learning libraries from tensorflow.
from tensorflow.keras.layers import GlobalMaxPooling2D, Dense,Flatten
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing import image

tf.__version__

'2.3.0'

In [ ]:
pip install -q pyyaml h5py

Using Pandas library we are able to input the csv file which contains the month and gender information regarding each image in dataset.

In [ ]:
#loading dataframes
train_df = pd.read_csv('/content/drive/MyDrive/BE_Project/Bone_Age_Detection/boneage-training-dataset.csv')
test_df = pd.read_csv('/content/drive/MyDrive/BE_Project/Bone_Age_Detection/boneage-test-dataset.csv')

#appending file extension to id column for both training and testing dataframes
train_df['id'] = train_df['id'].apply(lambda x: str(x)+'.png')

#train_df.head()
index = train_df.index
index2 = test_df.index
number_of_rows = len(index)
number_of_rows2 = len(index2)
print(number_of_rows)
print(number_of_rows2)
plt.bar("Train Data", number_of_rows)
plt.bar("Test Data", number_of_rows2)
plt.title('Dataset')
plt.xlabel('Datasets split')
plt.ylabel('Number of Images')
plt.show()

Here we find out the number of male and female children in the dataset.

In [ ]:
#creating a new column called gender to keep the gender of the child as a string
train_df['gender'] = train_df['male'].apply(lambda x: 'male' if x else 'female')
print(train_df['gender'].value_counts())
sns.countplot(x = train_df['gender'])

In [ ]:
#oldest child in the dataset
print('MAX age: ' + str(train_df['boneage'].max()) + ' months')

#youngest child in the dataset
print('MIN age: ' + str(train_df['boneage'].min()) + ' months')

#mean age is
mean_bone_age = train_df['boneage'].mean()
print('mean: ' + str(mean_bone_age))

#median bone age
print('median: ' +str(train_df['boneage'].median()))

#standard deviation of boneage
std_bone_age = train_df['boneage'].std()

#models perform better when features are normalised to have zero mean and unity standard deviation
#using z score for the training
train_df['bone_age_z'] = (train_df['boneage'] - mean_bone_age)/(std_bone_age)

print(train_df.head())

MAX age: 228 months
MIN age: 1 months
mean: 127.3207517246848
median: 132.0
         id  boneage   male  gender  bone_age_z
0  1377.png      180  False  female    1.279181
1  1378.png       12  False  female   -2.800269
2  1379.png       94  False  female   -0.809109
3  1380.png      120   True    male   -0.177766
4  1381.png       82  False  female   -1.100498


Here we use keras for generating train and test split of the images.

In [ ]:
#splitting train dataframe into traininng and validation dataframes
df_train, df_valid = train_test_split(train_df, test_size = 0.2, random_state = 0)

#reducing down the size of the image 
img_size = 256

train_data_generator = ImageDataGenerator(preprocessing_function = preprocess_input)
val_data_generator = ImageDataGenerator(preprocessing_function = preprocess_input)

#datagen=ImageDataGenerator(preprocessing_function = preprocess_input,validation_split=0.25)

In [ ]:
#train data generator
train_generator = train_data_generator.flow_from_dataframe(
    dataframe = df_train,
    directory = '/content/drive/MyDrive/BE_Project/Bone_Age_Detection/boneage-training-dataset/boneage-training-dataset',
    x_col= 'id',
    y_col= 'bone_age_z',
    batch_size = 32,
    shuffle = True,
    seed=42,
    validate_filenames=False,
    class_mode= 'raw',
    color_mode = 'rgb',
    target_size = (img_size, img_size))

#validation data generator
val_generator = val_data_generator.flow_from_dataframe(
    dataframe = df_valid,
    directory = '/content/drive/MyDrive/BE_Project/Bone_Age_Detection/boneage-training-dataset/boneage-training-dataset',
    x_col = 'id',
    y_col = 'bone_age_z',
    batch_size = 32,
    shuffle = True,
    seed=42,
    validate_filenames=False,
    class_mode = 'raw',
    color_mode = 'rgb',
    target_size = (img_size, img_size))

plt.bar("Training Data", 10088)
plt.bar("Validation Data", 2523)
plt.title('Dataset')
plt.xlabel('Dataset Split')
plt.ylabel('Number of Images')
plt.show()

Found 10088 non-validated image filenames.
Found 2523 non-validated image filenames.


In [ ]:
#test data generator. Commented for time being.
'''
test_data_generator = ImageDataGenerator(preprocessing_function = preprocess_input)

test_generator = test_data_generator.flow_from_directory(
    directory = '/content/drive/MyDrive/BE_Project/Bone_Age_Detection/boneage-test-dataset',
    shuffle = True,
    class_mode = None,
    color_mode = 'rgb',
    target_size = (img_size,img_size))


test_X, test_Y = next(val_data_generator.flow_from_dataframe( 
                            df_valid, 
                            directory = '/content/drive/MyDrive/BE Project/Bone Age Detection/boneage-training-dataset/boneage-training-dataset',
                            x_col = 'id',
                            y_col = 'bone_age_z', 
                            target_size = (img_size, img_size),
                            batch_size = 2523,
                            class_mode = 'other'
                            )) 

                            def plot_it(history):
    function to plot training and validation error
    fig, ax = plt.subplots( figsize=(20,10))
    ax.plot(history.history['mae_in_months'])
    ax.plot(history.history['val_mae_in_months'])
    plt.title('Model Error')
    plt.ylabel('error')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc='upper right')
    ax.grid(color='black')
    plt.show()
'''

This Function outputs the Mean Average Error(MAE) in months between the predicted value obtained from the model and the actual value from the Dataset. This function is then feed to the model which acts as a Metrics. 

In [ ]:
def mae_in_months(x_p, y_p):
    '''function to return mae in months'''
    return mean_absolute_error((std_bone_age*x_p + mean_bone_age), (std_bone_age*y_p + mean_bone_age)) 

Creating our basic model for Bone Age Detection.

In [ ]:
#Bone Age Detection Model

model_1 = tf.keras.applications.xception.Xception(input_shape = (img_size, img_size, 3),
                                           include_top = False,
                                           weights = 'imagenet')
model_1.trainable = True
model_2 = Sequential()
model_2.add(model_1)
model_2.add(GlobalMaxPooling2D())
model_2.add(Flatten())
model_2.add(Dense(10, activation = 'relu'))
model_2.add(Dense(1, activation = 'linear'))


#compile model
model_2.compile(loss ='mse', optimizer= 'adam', metrics = [mae_in_months] )

#model summary
model_2.summary()

Fitting the Created model.

In [ ]:
#Fitting the designed model.

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=val_generator.n//val_generator.batch_size

#print(STEP_SIZE_TRAIN, STEP_SIZE_VALID)

history = model_2.fit(train_generator,
                      steps_per_epoch = 315,
                      validation_data = val_generator,
                      validation_steps = 1,
                      epochs = 50)

As this process requires a lot of time we have the final output Screenshot.

In [ ]:
path_test = '/content/drive/MyDrive/BE_Project/Bone_Age_Detection/boneage-training-dataset/boneage-training-dataset/1377.png'

img = image.load_img(path_test, target_size=(256, 256))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

images = np.vstack([x])

print('Mean Age is {} months'.format(mean_bone_age))
print('Standard Deviation Age is {} months'.format(std_bone_age))
pred = mean_bone_age + std_bone_age*(model_2.predict(images, batch_size=1))
print('Age is {} months'.format(float(pred)))
print(model_2.predict(images, batch_size=1))
print('Age is {} months'.format(float(pred/12.0)))